In [9]:
//  Copyright (c) 2020 Patrick Diehl
//
//  SPDX-License-Identifier: BSL-1.0
//  Distributed under the Boost Software License, Version 1.0. (See accompanying
//  file LICENSE_1_0.txt or copy at http://www.boost.org/LICENSE_1_0.txt)

In [10]:
#include <iostream>
#include<run_hpx.cpp>
#include<hpx/include/lcos.hpp>
#include<hpx/include/parallel_for_loop.hpp>
#include<atomic>

# Numerical integration

The trapezoidal rule can be used to approximate the definite integral
\begin{align*}
\int\limits_a^b f(x) dx \approx \frac{h}{2} \sum\limits_{k=1}^N (f(x_{k-1}) + f(x_k))
\end{align*}
assuming a uniform grid in the interval $[a,b]$ with the grid size $h=\frac{b-a}{N}$.

### Define the function $f(x)=x^2$ to integrate

In [11]:
double f(double x){
    return x*x;
}

input_line_14:1:8: error: redefinition of 'f'
double f(double x){
       ^
input_line_5:1:8: note: previous definition is here
double f(double x){
       ^


### Define the integration interval

In [12]:
size_t N = 100;
double a = 0;
double b = 2;

double h = (b-a) / N;

input_line_14:2:9: error: redefinition of 'N'
 size_t N = 100;
        ^
input_line_6:2:9: note: previous definition is here
 size_t N = 100;
        ^


# Serial implementation of the integration

In [13]:
double area = 0; 

for(size_t i = 1; i <= N; i++){
    
    area += f(h*(i-1))+f(h*i);
    
}

area *= h/2;

std::cout << area << std::endl;

input_line_14:2:9: error: redefinition of 'area'
 double area = 0; 
        ^
input_line_7:2:9: note: previous definition is here
 double area = 0; 
        ^


## Parallel implementation I (parallel algorithms)

In [14]:
run_hpx([](){

double area = 0;
hpx::lcos::local::mutex m;

hpx::for_loop(
	hpx::execution::par, 
	0, 
	N,
	[&](boost::uint64_t i)
		{
            m.lock();
		    area += f(h*(i-1))+f(h*i);
            m.unlock();
		}
	);

area = area * h /2 ;

std::cout << area << std::endl;

});

1.36113e+33


(void) @0x7fd7bcf5ee48


## Parallel implementation II (futurization)

In [15]:
double integrate(size_t begin, size_t end,double h){
 
    double area = 0;
    
    
    for(size_t i = begin; i <= end; i++){
         area += f(h*(i-1))+f(h*i);
    } 
    
    return area;
}

input_line_15:1:8: error: redefinition of 'integrate'
double integrate(size_t begin, size_t end,double h){
       ^
input_line_10:1:8: note: previous definition is here
double integrate(size_t begin, size_t end,double h){
       ^


In [16]:
run_hpx([](){

auto f1 = hpx::async(integrate,1,49,h);
auto f2 = hpx::async(integrate,50,100,h);

std::cout << h/2*(f1.get() + f2.get()) << std::endl;

});

2.6668


(void) @0x7fd7bcf5ee48
